<a href="https://colab.research.google.com/github/ehsanbarkhordar/Bank-Customer-Clustering-By-Transactions/blob/main/Czech_customer_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from keras.layers import Bidirectional

import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# Import transactions
path='/content/gdrive/MyDrive/lpetrocelli-retail-banking-demo-data/completedtrans.csv'
df_completedtrans = pd.read_csv(path, low_memory=False)
df_completedtrans = df_completedtrans.loc[:, ~df_completedtrans.columns.str.contains('^Unnamed')]
df_completedtrans

,trans_id,account_id,type,operation,amount,balance,k_symbol,bank,account,year,month,day,fulldate,fulltime,fulldatewithtime
0,T00695247,A00002378,Credit,Credit in Cash,700.0,700.0,NaN,NaN,NaN,2013,1,1,2013-01-01,11:02:40,2013-01-01T11:02:40
1,T00171812,A00000576,Credit,Credit in Cash,900.0,900.0,NaN,NaN,NaN,2013,1,1,2013-01-01,08:23:33,2013-01-01T08:23:33
2,T00207264,A00000704,Credit,Credit in Cash,1000.0,1000.0,NaN,NaN,NaN,2013,1,1,2013-01-01,15:19:21,2013-01-01T15:19:21
3,T01117247,A00003818,Credit,Credit in Cash,600.0,600.0,NaN,NaN,NaN,2013,1,1,2013-01-01,10:42:35,2013-01-01T10:42:35
4,T00579373,A00001972,Credit,Credit in Cash,400.0,400.0,NaN,NaN,NaN,2013,1,2,2013-01-02,11:39:25,2013-01-02T11:39:25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1056315,T03626622,A00002906,Credit,NaN,62.3,13729.4,Interest Credited,NaN,NaN,2018,12,31,2018-12-31,15:18:14,2018-12-31T15:18:14
1056316,T03627616,A00002935,Credit,NaN,81.3,19544.9,Interest Credited,NaN,NaN,2018,12,31,2018-12-31,13:04:60,2018-12-31T13:04:60
1056317,T03625403,A00002869,Credit,NaN,60.2,14638.2,Interest Credited,NaN,NaN,2018,12,31,2018-12-31,08:30:13,2018-12-31T08:30:13
1056318,T03626683,A00002907,Credit,NaN,107.5,23453.0,Interest Credited,NaN,NaN,2018,12,31,2018-12-31,14:45:19,2018-12-31T14:45:19


In [ ]:
# Convert date to python datetime
df_completedtrans['fulldate'] = pd.to_datetime(df_completedtrans['fulldate'],format="%Y-%m-%d")

In [ ]:
df_grouped=df_completedtrans.groupby(['account_id'])

In [ ]:
X1 = np.array([x+3 for x in range(0, 135, 3)])
print(X1)

X2 = np.array([x+5 for x in range(0, 225, 5)])
print(X2)

NameError: ignored

In [ ]:
X = np.column_stack((X1, X2))
print(X)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45]


In [ ]:
X = X.reshape(15,3,1)
print(X)

[[[ 1]
  [ 2]
  [ 3]]

 [[ 4]
  [ 5]
  [ 6]]

 [[ 7]
  [ 8]
  [ 9]]

 [[10]
  [11]
  [12]]

 [[13]
  [14]
  [15]]

 [[16]
  [17]
  [18]]

 [[19]
  [20]
  [21]]

 [[22]
  [23]
  [24]]

 [[25]
  [26]
  [27]]

 [[28]
  [29]
  [30]]

 [[31]
  [32]
  [33]]

 [[34]
  [35]
  [36]]

 [[37]
  [38]
  [39]]

 [[40]
  [41]
  [42]]

 [[43]
  [44]
  [45]]]


In [ ]:
Y = list()
for x in X:
    Y.append(x.sum())

Y = np.array(Y)
print(Y)

[  6  15  24  33  42  51  60  69  78  87  96 105 114 123 132]


In [ ]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(3, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1)

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 4072.7585 - val_loss: 15174.4951
Epoch 2/1000
1/1 [==============================] - 0s 39ms/step - loss: 4060.1672 - val_loss: 15137.6836
Epoch 3/1000
1/1 [==============================] - 0s 36ms/step - loss: 4047.8184 - val_loss: 15100.9971
Epoch 4/1000
1/1 [==============================] - 0s 38ms/step - loss: 4035.6409 - val_loss: 15064.3076
Epoch 5/1000
1/1 [==============================] - 0s 43ms/step - loss: 4023.5791 - val_loss: 15027.4580
Epoch 6/1000
1/1 [==============================] - 0s 42ms/step - loss: 4011.5801 - val_loss: 14990.2998
Epoch 7/1000
1/1 [==============================] - 0s 48ms/step - loss: 3999.5945 - val_loss: 14952.6846
Epoch 8/1000
1/1 [==============================] - 0s 39ms/step - loss: 3987.5742 - val_loss: 14914.4844
Epoch 9/1000
1/1 [==============================] - 0s 44ms/step - loss: 3975.4812 - val_loss: 14875.6133
Epoch 10/1000
1/1 [=============================

In [ ]:
test_input = array([50,51,52])
test_input = test_input.reshape((1, 3, 1))
test_output = model.predict(test_input, verbose=0)
print(test_output)

[[152.05026]]


In [ ]:
model = Sequential()
model.add(LSTM(200, activation='relu', return_sequences=True, input_shape=(3, 1)))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(LSTM(50, activation='relu', return_sequences=True))
model.add(LSTM(25, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1)

Epoch 1/1000
1/1 [==============================] - 4s 4s/step - loss: 4045.1111 - val_loss: 15177.0391
Epoch 2/1000
1/1 [==============================] - 0s 49ms/step - loss: 4043.8196 - val_loss: 15172.2627
Epoch 3/1000
1/1 [==============================] - 0s 48ms/step - loss: 4042.4558 - val_loss: 15165.9453
Epoch 4/1000
1/1 [==============================] - 0s 50ms/step - loss: 4040.7529 - val_loss: 15157.0127
Epoch 5/1000
1/1 [==============================] - 0s 46ms/step - loss: 4038.6433 - val_loss: 15144.4873
Epoch 6/1000
1/1 [==============================] - 0s 54ms/step - loss: 4035.9338 - val_loss: 15126.5312
Epoch 7/1000
1/1 [==============================] - 0s 51ms/step - loss: 4032.3259 - val_loss: 15100.1318
Epoch 8/1000
1/1 [==============================] - 0s 48ms/step - loss: 4027.3713 - val_loss: 15060.0244
Epoch 9/1000
1/1 [==============================] - 0s 52ms/step - loss: 4020.3811 - val_loss: 14999.2139
Epoch 10/1000
1/1 [=============================

In [ ]:
test_output = model.predict(test_input, verbose=0)
print(test_output)

[[153.7054]]
